In [1]:
'''
https://www.cnblogs.com/earendil/p/8194875.html
http://www.shuang0420.com/2017/02/17/%E6%8E%A8%E8%8D%90%E7%B3%BB%E7%BB%9F--%E9%9A%90%E8%AF%AD%E4%B9%89%E6%A8%A1%E5%9E%8BLFM/
'''

'\nhttps://www.cnblogs.com/earendil/p/8194875.html\nhttp://www.shuang0420.com/2017/02/17/%E6%8E%A8%E8%8D%90%E7%B3%BB%E7%BB%9F--%E9%9A%90%E8%AF%AD%E4%B9%89%E6%A8%A1%E5%9E%8BLFM/\n'

In [2]:
'''
lfm召回算法
'''

'\nlfm召回算法\n'

In [3]:
# 初始化需要用到
import numpy as np
# 从util引入获取训练集的函数
import sys
sys.path.append(r"F:\\FictionRecommend\\")
import Ipynb_importer
import read
# print(read)
# 排序
import operator

importing Jupyter notebook from read.ipynb
67186
[('1', '1', 1), ('1', '3', 1), ('1', '6', 1), ('1', '47', 1), ('1', '50', 1), ('1', '101', 1), ('1', '110', 1), ('1', '151', 1), ('1', '157', 1), ('1', '163', 1), ('1', '216', 1), ('1', '231', 1), ('1', '235', 1), ('1', '260', 1), ('1', '333', 1), ('1', '349', 1), ('1', '356', 1), ('1', '362', 1), ('1', '367', 1), ('1', '441', 1), ('1', '457', 1), ('1', '480', 1), ('1', '527', 1), ('1', '543', 1), ('1', '552', 1), ('1', '553', 1), ('1', '590', 1), ('1', '592', 1), ('1', '593', 1), ('1', '596', 1), ('1', '608', 1), ('1', '661', 1), ('1', '296', 0), ('1', '1258', 0), ('1', '1219', 0), ('1', '223', 0), ('1', '1408', 0), ('1', '648', 0), ('1', '70', 0), ('1', '1580', 0)]


In [4]:
movies_file = "F:\\badou\\tmp\\data\\recommender\\data\\ml-latest-small\\movies.csv"
ratings_file = "F:\\badou\\tmp\\data\\recommender\\data\\ml-latest-small\\ratings.csv"

In [5]:
def lfm_train(train_data, F, alpha, beta, step):
    '''
    lfm 模型的训练
    :param train_data:
        训练集
    :param F:
        维度(分类数)[用户向量和item向量的长度(维度)]
    :param alpha:
        正则化参数
    :param beta:
        学习率
    :param step:
        迭代次数
    :return:
        两个 np.ndarray:
            user向量
            item向量
    '''
    user_vec = {}
    item_vec = {}
    
    # 迭代
    for step_index in range(step):
        # 从训练样本中获取每一个实例
        for data_instance in train_data:
            user_id, item_id, label = data_instance
            # 第一次训练初始化(初始化为长度为F的向量)
            if user_id not in user_vec:
                user_vec[user_id] = init_model(F)
            if item_id not in item_vec:
                item_vec[item_id] = init_model(F)
        # 模型预测
        delta = label - model_predict(user_vec[user_id], item_vec[item_id])
        # 模型迭代参数
        for index in range(F):
            # 对user_vec的每一个维度(index)也就是f
            # 相当于公式中西格玛下面的 f = 1
            # loss函数对于 pu 小 f 求偏导
            user_vec[user_id][index] += beta * (delta * item_vec[item_id][index] - alpha * user_vec[user_id][index])
            # pi 小 f
            item_vec[item_id][index] += beta * (delta * user_vec[user_id][index] - alpha * item_vec[item_id][index])
        # 每一轮迭代在参数更加接近收敛时使参数变换的慢一些
        beta = beta * 0.9
    return user_vec, item_vec

In [6]:
# 按照标准的正态分布进行初始化
def init_model(vector_len):
    '''
    :param vector_len:
        F
    :return:
        ndarray的数据结构
    '''
    return np.random.randn(vector_len)

In [7]:
def model_predict(user_vector, item_vector):
    '''
    :param user_vector:
        模型产出的用户向量
    :param item_vector:
        模型产出的item向量
    :return:
        返回一个 num
            表示user_vec和item_vec之间的远近(即模型认为的推荐强度)
    '''
    # 使用余弦来表示距离
    # dot 内积
    res = np.dot(user_vector, item_vector) / (np.linalg.norm(user_vector) * np.linalg.norm(item_vector))
    return res

In [8]:
def give_recom_result(user_vec, item_vec, user_id):
    '''
    :param user_vec:
    :param item_vec:
        模型的结果
    :param user_id:
        推荐的结果
    :return:
        list[(item_id, score), (item_id1, score)]
    '''
    topN = 10

    if user_id not in user_vec:
        return []
    
    # 存储每一个item和user_vec之间的距离
    record = {}
    recom_list = []
    user_vector = user_vec[user_id]
    
    for item_id in item_vec:
        # 每一个item_id和user_id对应的向量(vec)之间的距离
        item_vector = item_vec[item_id]
        # 用欧氏距离计算
        res = np.dot(user_vector, item_vector) / (np.linalg.norm(user_vector) * np.linalg.norm(item_vector))
        record[item_id] = res
        
    # topn 推荐
    for zuhe in sorted(record.items(), key = operator.itemgetter(1), reverse = True)[:topN]:
        item_id = zuhe[0]
        # 3位有效数字
        score = round(zuhe[1], 3)
        recom_list.append((item_id, score))
        
        
    return recom_list

In [9]:
def ana_recom_result(train_data, user_id, recom_list):
    """测试，判断推荐结果的准确性"""
    item_info = read.get_item_info(movies_file)
    # 标签数据
    for data_instance in train_data:
        tmp_user_id, item_id, label = data_instance
        if tmp_user_id == user_id and label == 1:
            print(item_info[item_id], " ", item_id)
    print("recom list" + "***" * 30)
    
    # 推荐结果
    for zuhe in recom_list:
        print(item_info[zuhe[0]], " ", zuhe[0])

In [10]:
def model_train_process():
    '''
    :return:
    '''
    # 训练数据
    train_data = read.get_train_data(ratings_file)
    user_vec, item_vec = lfm_train(train_data, 50, 0.01, 0.1, 50)
    # userid, itemid 1,648
    # print(user_vec["1"])
    # print(item_vec["648"])

    recom_list = give_recom_result(user_vec, item_vec, "24")
    ana_recom_result(train_data, "24", recom_list)

#     for user_id in user_vec:
#         recom_list = give_recom_result(user_vec, item_vec, user_id)

In [11]:
model_train_process()

['Heat (1995)', 'Action|Crime|Thriller']   6
['"Usual Suspects, The (1995)"', 'Crime|Mystery|Thriller']   50
['Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller']   296
['"Shawshank Redemption, The (1994)"', 'Crime|Drama']   318
['Forrest Gump (1994)', 'Comedy|Drama|Romance|War']   356
['"Silence of the Lambs, The (1991)"', 'Crime|Horror|Thriller']   593
['Fargo (1996)', 'Comedy|Crime|Drama|Thriller']   608
['"Princess Bride, The (1987)"', 'Action|Adventure|Comedy|Fantasy|Romance']   1197
['Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'Action|Adventure']   1198
['Dead Poets Society (1989)', 'Drama']   1246
['Groundhog Day (1993)', 'Comedy|Fantasy|Romance']   1265
['Sneakers (1992)', 'Action|Comedy|Crime|Drama|Sci-Fi']   1396
['"Fifth Element, The (1997)"', 'Action|Adventure|Comedy|Sci-Fi']   1527
['Men in Black (a.k.a. MIB) (1997)', 'Action|Comedy|Sci-Fi']   1580
['Good Will Hunting (1997)', 'Drama|Romance']   1704
['As Good as It Gets (1997)', 'Com